In [1]:
from huggingface_hub import login
# Log in with your API token
login(token="YOURTOKENHERE")
import transformers
import torch
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Set CUDA deterministic mode
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define the device
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

def load_model(model_id="meta-llama/Meta-Llama-3-8B", device="cuda"):
    # Load the model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    return model, tokenizer
model, tokenizer = load_model(device=device)

/mnt/data3/swu/miniforge3/envs/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|█████████████████████████████████████████████████| 4/4 [00:00<00:00,  6.44it/s]


# grafting

In [2]:
# Define the input text and the target token
input_text = "Hi, how are you doing?"
# Tokenize input and identify the token position to analyze
input_tokens = tokenizer(input_text, return_tensors="pt").to(device)

In [5]:
import torch
import numpy as np
import random

# Set seeds for reproducibility
def set_seed(seed=42):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Set the seed
set_seed(42)

# Generate text before perturbation
attention_mask = input_tokens["attention_mask"]

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token if pad_token is not set

output_before = model.generate(
    input_ids=input_tokens["input_ids"],
    attention_mask=attention_mask,                # Provide the attention mask
    max_length=30,
    pad_token_id=tokenizer.pad_token_id,          # Explicitly set pad token ID
    temperature=0.0,                              # Set temperature to 0 for deterministic output
    do_sample=False,
    num_beams=1,# Disable sampling to ensure deterministic behavior
)

print("Before Perturbation:", tokenizer.decode(output_before[0], skip_special_tokens=True))


Before Perturbation: What shall we make today?  I have a few ideas, but I’m open to suggestions.  I’m thinking of making a


In [ ]:
import pickle

with open("./neural_chunk_dictionary/neural_chunk_dictionary.pkl", "rb") as file:
    neural_chunk_dictionary = pickle.load(file)

hook_counter = 0

def create_modify_input_hook(constant_values=None, constant_positions=None, token_idx=3):
    """Returns a hook function that modifies the input during the forward pass."""
    
    def modify_input_hook(module, input):
        """
        Hook to modify the input during the forward pass.
        Args:
            module: The layer being hooked.
            input: The inputs to the layer.
        Returns:
            Modified input.
        """
        modified_input = input[0].clone() # shape = 1 x 8 x 4096
        
        if modified_input.shape[1]>1:
            modified_input[:, token_idx, constant_positions] = constant_values #shape  torch.Size([1, 8, 4096])
            
        global hook_counter
        hook_counter += 1
        return (modified_input,) # I don't understand why the second time the modified input takes the shape of 1 x 4096
        
    return modify_input_hook

word = 'cake'
print('grafting word = ', word)

# Register pre-hooks on specific layers
hooks = []  # To store references to the hooks
token_idx = 6

for layer_to_perturb in range(2,31):  
    # Register the pre-hook on the forward pass of the layer
    constant_values = neural_chunk_dictionary[word]['layer'][layer_to_perturb]['constant_values']
    constant_positions = neural_chunk_dictionary[word]['layer'][layer_to_perturb]['constant_positions'][:,1] # perturb so the network thinks that it is a cake, instead of cheese cake 
    #print(constant_values)
    #print(constant_positions)
    constant_positions_tensor = torch.tensor(constant_positions, dtype=torch.int64, device=device)
    constant_values_tensor = torch.tensor(constant_values,dtype=torch.bfloat16, device=device)

    transformer_layers = model.model.layers[layer_to_perturb].input_layernorm.to(device)
    hook = transformer_layers.register_forward_pre_hook(create_modify_input_hook(constant_values=constant_values_tensor, constant_positions=constant_positions_tensor, token_idx=token_idx))
    hooks.append(hook)
    
inputs = input_tokens.to(device)

with torch.no_grad():
    output_after = model.generate(input_tokens['input_ids'], 
                                  max_length=200,
                                  attention_mask=attention_mask, 
                                  pad_token_id=tokenizer.pad_token_id)
    print(len(output_after[0]))
    print("After Perturbation:", tokenizer.decode(output_after[0], skip_special_tokens=True))

# Remove all hooks
for hook in hooks: hook.remove()
# # Access the modified hidden states
# hidden_states = outputs.hidden_states

# freezing 

In [15]:
# Define the input text and the target token
input_text = "What is the name of a rich, savory food made from curdled milk, often aged to enhance its flavors? "
# Tokenize input and identify the token position to analyze
input_tokens = tokenizer(input_text, return_tensors="pt").to(device)

In [21]:
# Generate text before perturbation
attention_mask = input_tokens["attention_mask"]

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token if pad_token is not set
# Generate output with the correct pad_token_id and attention_mask
output_before = model.generate(
    input_ids=input_tokens["input_ids"],
    attention_mask=attention_mask,                # Provide the attention mask
    max_length=200,
    pad_token_id=tokenizer.pad_token_id,           # Explicitly set pad token ID
    temperature=0.0,                              # Set temperature to 0 for deterministic output
    do_sample=False,
    num_beams=1,# Disable sampling to ensure deterministic behavior
)
print("Before Perturbation:", tokenizer.decode(output_before[0], skip_special_tokens=True))


Before Perturbation: What is the name of a rich, savory food made from curdled milk, often aged to enhance its flavors?  It is a staple of many cuisines, and is often used as a condiment or ingredient in other dishes.  It is also a popular ingredient in many desserts.  What is it?
The answer is cheese.  Cheese is a dairy product made from the curdled milk of various animals, including cows, goats, and sheep.  It is a staple of many cuisines, and is often used as a condiment or ingredient in other dishes.  It is also a popular ingredient in many desserts.
Cheese is made by curdling milk with an acid, such as lemon juice or vinegar.  The curds are then separated from the whey, and the curds are pressed into blocks or wheels.  The curds are then aged to enhance their flavors.  The aging process can take


In [ ]:
import pickle

with open("./neural_chunk_dictionary/neural_chunk_dictionary.pkl", "rb") as file:
    neural_chunk_dictionary = pickle.load(file)

hook_counter = 0

def create_modify_input_hook(constant_values=None, constant_positions=None, token_idx=3):
    """Returns a hook function that modifies the input during the forward pass."""
    
    def modify_input_hook(module, input):
        """
        Hook to modify the input during the forward pass.
        Args:
            module: The layer being hooked.
            input: The inputs to the layer.
        Returns:
            Modified input.
        """
        modified_input = input[0].clone() # shape = 1 x 8 x 4096
        
        if modified_input.shape[1]>1:
            modified_input[:, token_idx, constant_positions] = 0 
            
        global hook_counter
        hook_counter += 1
        return (modified_input,) # I don't understand why the second time the modified input takes the shape of 1 x 4096
        
    return modify_input_hook

word = 'cheese'
print('word = ', word)

# Register pre-hooks on specific layers
hooks = []  # To store references to the hooks
token_idx = 7

for layer_to_perturb in range(1,32):  
    # Register the pre-hook on the forward pass of the layer
    constant_values = neural_chunk_dictionary[word]['layer'][layer_to_perturb]['constant_values']
    constant_positions = neural_chunk_dictionary[word]['layer'][layer_to_perturb]['constant_positions'][:,1] # perturb so the network thinks that it is a cake, instead of cheese cake 

    constant_positions_tensor = torch.tensor(constant_positions, dtype=torch.int64, device=device)
    constant_values_tensor = torch.tensor(constant_values,dtype=torch.bfloat16, device=device)

    transformer_layers = model.model.layers[layer_to_perturb].input_layernorm.to(device)
    hook = transformer_layers.register_forward_pre_hook(create_modify_input_hook(constant_values=constant_values_tensor, constant_positions=constant_positions_tensor, token_idx=token_idx))
    hooks.append(hook)
    
inputs = input_tokens.to(device)

with torch.no_grad():
    output_before = model.generate(
    input_ids=input_tokens["input_ids"],
    attention_mask=attention_mask,                # Provide the attention mask
    max_length=200,
    pad_token_id=tokenizer.pad_token_id,          # Explicitly set pad token ID
    temperature=0.0,                              # Set temperature to 0 for deterministic output
    do_sample=False,
    num_beams=1,# Disable sampling to ensure deterministic behavior
)
    print(len(output_after[0]))
    print("After Perturbation:", tokenizer.decode(output_after[0], skip_special_tokens=True))

# Remove all hooks
for hook in hooks: hook.remove()
# # Access the modified hidden states
# hidden_states = outputs.hidden_states